In [ ]:
##3 버전에러 해결

In [1]:
%matplotlib inline


# Speech Command Classification with torchaudio

This tutorial will show you how to correctly format an audio dataset and
then train/test an audio classifier network on the dataset.

Colab has GPU option available. In the menu tabs, select “Runtime” then
“Change runtime type”. In the pop-up that follows, you can choose GPU.
After the change, your runtime should automatically restart (which means
information from executed cells disappear).

First, let’s import the common torch packages such as
[torchaudio](https://github.com/pytorch/audio)_ that can be installed
by following the instructions on the website.


In [15]:
# Uncomment the line corresponding to your "runtime type" to run in Google Colab

# CPU:
!pip install pydub torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# GPU:
# !pip install pydub torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached https://download.pytorch.org/whl/cpu/torch-1.7.0%2Bcpu-cp38-cp38-linux_x86_64.whl (159.4 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.8.1%2Bcpu-cp38-cp38-linux_x86_64.whl (11.8 MB)
  Using cached torchaudio-0.7.0-cp38-cp38-manylinux1_x86_64.whl (7.6 MB)
  Using cached future-0.18.3.tar.gz (840 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
     |████████████████████████████████| 3.5 MB 5.5 MB/s 
Using legacy 'setup.py install' for future, since package 'wheel' is not installed.
    Running setup.py install for future ... done


In [14]:
pip install --upgrade setuptools  #command errored out with exit status 1 python setup.py egg_info 오류 해결

     |████████████████████████████████| 807 kB 2.8 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 49.2.1
    Uninstalling setuptools-49.2.1:
      Successfully uninstalled setuptools-49.2.1


In [3]:
print(torch.__version__)
!python -V            # 버전확인

2.1.0+cu118
Python 3.10.12


In [4]:
#파이썬 버전 변경 3.10.12 -> 3.8.6
#ERROR: Could not find a version that satisfies the requirement torch==1.7.0+cpu (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
#ERROR: No matching distribution found for torch==1.7.0+cpu 해결
!wget https://www.python.org/ftp/python/3.8.6/Python-3.8.6.tgz
!tar xvfz Python-3.8.6.tgz
!Python-3.8.6/configure
!make
!sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.8/importlib/_bootstrap_external.py'...
Compiling '/usr/local/lib/python3.8/importlib/abc.py'...
Compiling '/usr/local/lib/python3.8/importlib/machinery.py'...
Compiling '/usr/local/lib/python3.8/importlib/metadata.py'...
Compiling '/usr/local/lib/python3.8/importlib/resources.py'...
Compiling '/usr/local/lib/python3.8/importlib/util.py'...
Compiling '/usr/local/lib/python3.8/inspect.py'...
Compiling '/usr/local/lib/python3.8/io.py'...
Compiling '/usr/local/lib/python3.8/ipaddress.py'...
Compiling '/usr/local/lib/python3.8/json/__init__.py'...
Compiling '/usr/local/lib/python3.8/json/decoder.py'...
Compiling '/usr/local/lib/python3.8/json/encoder.py'...
Compiling '/usr/local/lib/python3.8/json/scanner.py'...
Compiling '/usr/local/lib/python3.8/json/tool.py'...
Compiling '/usr/local/lib/python3.8/keyword.py'...
Compiling '/usr/local/lib/python3.8/lib2to3/__init__.py'...
Compiling '/usr/local/lib/python3.8/lib2to3/__mai

Let’s check if a CUDA GPU is available and select our device. Running
the network on a GPU will greatly decrease the training/testing runtime.




In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu
